<h1>2. fáza - predspracovanie dát</h1>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import re

In [ ]:
def splitPersonalInfo(dataO):
    temp = dataO['personal_info'].str.split("\r\r\n|--|\\|", expand = True)
    dataO['ethn'] = temp[0]
    dataO['state'] = temp[1]
    dataO['edu'] = temp[2]
    dataO['status'] = temp[3]
    dataO['fam_memb'] = temp[4] 
    dataO.drop(['personal_info'], axis = 1, inplace = True)

In [ ]:
def fusionDuplicates(datasetO):
    duplicat = datasetO[datasetO['name'].duplicated(keep=False)]
    duplicat = duplicat.sort_values('name')
    duplicat = duplicat.reset_index(drop=True)
    
    for i, row in duplicat.iterrows():
        if(i%2==1):
            duplicat = duplicat.drop(i)
            continue
        for j, value in row.items():
            dupValue = duplicat.loc[i+1,j]
            if(pd.isna(value)):
                duplicat.loc[i,j]=dupValue
    datasetO.drop_duplicates(subset='name',keep=False,inplace=True)
    result = datasetO.append(duplicat, ignore_index=True)
    return(result)

In [ ]:
def correctValuesO(dataset):
    for col in dataset.columns:
        if(dataset[col].dtype != np.object):
           continue
        dataset[col] = dataset[col].str.replace('_','-')
        dataset[col] = dataset[col].replace(to_replace = r'.*\?.*', value = np.nan, regex = True)
        dataset[col] = dataset[col].replace(to_replace = r'[Nn][Aa][Nn]', value = np.nan, regex = True)
        dataset[col] = dataset[col].replace(['T','F'], ['t', 'f'])
        dataset[col] = dataset[col].replace(to_replace = r'[Tt][Rr][Uu][Ee]', value = 't', regex = True)
        dataset[col] = dataset[col].replace(to_replace = r'[Ff][Aa][Ll][Ss][Ee]', value = 'f', regex = True)

def correctDates(series):
    s = series.str.split("-", 3, expand = True)
    x = 0
    while(x<len(series)):
        if(re.match(r'^[0-9][0-9]-[0-9][0-9]-[0-9][0-9]$', series.loc[x])):
            if(s.iloc[x][0][:1]=='0'):
                series.loc[x] = "20"+s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2]
            else:
                series.loc[x] = "19"+s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2]
        if(len(s.iloc[x][2])>2):
            series.loc[x] = s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2][:2]
        if(re.match(r'^[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]$', series.loc[x])):
            y = s.iloc[x][0]
            s.iloc[x][0] = s.iloc[x][2]
            s.iloc[x][2] = y
            series.loc[x] = s.iloc[x][0]+"-"+s.iloc[x][1]+"-"+s.iloc[x][2]
        x += 1
        print(x)
        
def correctValuesP(dataset):
    for col in dataset.columns:
        if(dataset[col].dtype != np.object):
           continue
        dataset[col] = dataset[col].str.replace('/','-')    
    correctDates(dataset["date_of_birth"])
        

In [ ]:
dataP = pd.read_csv("personal_train.csv", index_col=0)
dataO = pd.read_csv("other_train.csv", index_col=0)

In [ ]:
splitPersonalInfo(dataO)

In [ ]:
dataP.info()

In [ ]:
dataO.info()

In [ ]:
len(dataO.name.unique())

In [ ]:
namesO = dataO["name"].unique().sort()

In [ ]:
namesP = dataP["name"].unique().sort()

In [ ]:
np.array_equal(namesO, namesP)

In [ ]:
dataO.head()

In [ ]:
    dataOtypes = dataO["kurtosis_oxygen"].str.isdecimal()

In [ ]:
dataOtypes.shape

In [ ]:
dataOtypes.head()

In [ ]:
duplNames = dataO[dataO['name'].duplicated(keep=False)]
duplNames.sort_values('name')

Po vypísani duplicitných mien sme zistili, že zlúčením týchto duplikátov vieme vyplniť chýbajúce hodnoty. Zároveň hodnoty, ktoré majú oba duplikáty sú rovnaké. 

In [ ]:
dataO=fusionDuplicates(dataO)
dataO

In [ ]:
dataO.info()

In [ ]:
dataO[dataO.isnull().any(axis=1)]

In [ ]:
replaced = dataO.replace(to_replace=r'^[0-9,\-].*[0-9]$',value='number',regex=True)
replaced.select_dtypes(include='object')

In [ ]:
replaced['kurtosis_oxygen'].unique()

In [ ]:
replaced['occupation'].unique()

In [ ]:
replaced['pregnant'].unique()

In [ ]:
replaced['education-num'].unique()

In [ ]:
replaced['edu'].unique()

In [ ]:
replaced['income'].unique()

In [ ]:
replaced['workclass'].unique()

In [ ]:
replaced['ethn'].unique()

In [ ]:
replaced['state'].unique()

In [ ]:
replaced['status'].unique()

In [ ]:
replaced['fam_memb'].unique()

In [ ]:
correctValuesO(dataO)

In [ ]:
dataO['pregnant'].unique()

In [ ]:
dataP.head(26)

In [ ]:
replaced = dataP.replace(to_replace = r'[0-9]', value = 'x', regex = True)

In [ ]:
replaced["date_of_birth"].unique()

xx/xx/xxxx --> dd/mm/yyyy

In [ ]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]')].sort_values(ascending=False).str.split("/", 3, expand = True)

In [ ]:
date.max()

xxxx/xx/xx --> yyyy/mm/dd

In [ ]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9][0-9][0-9]/[0-9][0-9]/[0-9][0-9]')].sort_values(ascending=False).str.split("/", 3, expand = True)

In [ ]:
date.max()

xxxx-xx-xx --> yyyy-mm-dd

In [ ]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]')].sort_values(ascending=False).str.split("-", 3, expand = True)

In [ ]:
date.max()

xx-xx-xx --> yy-mm-dd

In [ ]:
date = dataP["date_of_birth"][dataP["date_of_birth"].str.match(r'[0-9][0-9]-[0-9][0-9]-[0-9][0-9]')].sort_values(ascending=False).str.split("-", 3, expand = True)

In [ ]:
date.max()

In [ ]:
date[0].unique()

In [ ]:
correctValuesP(dataP)

In [ ]:
dataP['date_of_birth']

In [ ]:
replaced = dataP.replace(to_replace = r'[0-9]', value = 'x', regex = True)
replaced['date_of_birth'].unique()